In [1]:
import findspark
findspark.init('D:\spark\spark-3.3.2-bin-hadoop2/')
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [3]:
mongo_ip = 'mongodb://localhost:27017/Cleaned_Insurance'

In [4]:
spark = SparkSession \
.builder\
.appName("myApp1")\
.config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
.getOrCreate()

In [5]:
b_08 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2008").load()

In [6]:
b_09 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2009").load()

In [7]:
b_10 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2010").load()

In [8]:
Inp =  spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".Inpatient").load()

In [9]:
b_08.createOrReplaceTempView('b_08')
b_09.createOrReplaceTempView('b_09')
b_10.createOrReplaceTempView('b_10')
Inp.createOrReplaceTempView("Inp")

In [10]:
b_09.columns

['Age',
 'Alzheimer',
 'Cancer',
 'Chronic_Kidney_Disease',
 'Chronic_Obstructive_Pulmonary_Disease',
 'DESYNPUF_ID',
 'Date_of_birth',
 'Date_of_death',
 'Depression',
 'Diabetes',
 'End_stage_renal_disease_Indicator',
 'Gender',
 'Heart_Failure',
 'Inpatient_primary_payer_reimbursement_amount',
 'Inpatient_reimbursement_amount',
 'Inpatient_responsibility_amount',
 'Ischemic_Heart_Disease',
 'Osteoporosis',
 'Outpatient_primary_payer_reimbursement_amount',
 'Outpatient_reimbursement_amount',
 'Outpatient_responsibility_amount',
 'Race_Code',
 'Rheumatoid_arthritis_osteoarthritis',
 'State_Code',
 'Stroke_transient_Ischemic_Attack',
 'Total_months_HMO_coverage',
 'Total_months_partA_coverage',
 'Total_months_partB_coverage',
 'Total_months_partD_coverage',
 '_id',
 'total_number_of_Insurance',
 'total_number_of_diseases',
 'year']

In [11]:
b_08 = spark.sql('select * from b_08 where Date_of_death is null')
b_09 = spark.sql('select * from b_09 where Date_of_death is null')
b_10 = spark.sql('select * from b_10 where Date_of_death is null')

In [12]:
col = ['Age',
 'Alzheimer',
 'Cancer',
 'Chronic_Kidney_Disease',
 'Chronic_Obstructive_Pulmonary_Disease',
 'Depression',
 'Diabetes',
 'End_stage_renal_disease_Indicator',
 'Gender',
 'Heart_Failure',
 'Inpatient_reimbursement_amount',
 'Ischemic_Heart_Disease',
 'Osteoporosis',
 'Rheumatoid_arthritis_osteoarthritis',
 'Stroke_transient_Ischemic_Attack',
 'DESYNPUF_ID'
      ]

In [13]:
b_08 = b_08.select(col)
b_09 = b_09.select(col)
b_10 = b_10.select(col)

In [14]:
from pyspark.sql.functions import col,when
b_08 = b_08.withColumn("End_stage_renal_disease_Indicator",when(col("End_stage_renal_disease_Indicator") == 0 ,'F').otherwise('T'))
b_09 = b_09.withColumn("End_stage_renal_disease_Indicator",when(col("End_stage_renal_disease_Indicator") == 0 ,'F').otherwise('T'))
b_10 = b_10.withColumn("End_stage_renal_disease_Indicator",when(col("End_stage_renal_disease_Indicator") == 0 ,'F').otherwise('T'))

In [15]:
b_08 = b_08.withColumn("Inpatient_reimbursement_amount",b_08["Inpatient_reimbursement_amount"].cast('int'))
b_09 = b_09.withColumn("Inpatient_reimbursement_amount",b_09["Inpatient_reimbursement_amount"].cast('int'))
b_10 = b_10.withColumn("Inpatient_reimbursement_amount",b_10["Inpatient_reimbursement_amount"].cast('int'))

In [16]:
ch_col = ['Alzheimer',
 'Cancer',
 'Chronic_Kidney_Disease',
 'Chronic_Obstructive_Pulmonary_Disease',
 'Depression',
 'Diabetes','Heart_Failure', 'Ischemic_Heart_Disease',
 'Osteoporosis',
 'Rheumatoid_arthritis_osteoarthritis',
 'Stroke_transient_Ischemic_Attack']

In [17]:
for col_name in ch_col:
    b_08 = b_08.withColumn(col_name,b_08[col_name].cast('string'))

In [18]:
for col_name in ch_col:
    b_09 = b_09.withColumn(col_name,b_09[col_name].cast('string'))

In [19]:
for col_name in ch_col:
    b_10 = b_10.withColumn(col_name,b_10[col_name].cast('string'))

In [20]:
for col_name in ch_col:
    b_08 = b_08.withColumn(col_name,when(col(col_name) == False ,'F').otherwise('T'))

In [21]:
for col_name in ch_col:
    b_09 = b_09.withColumn(col_name,when(col(col_name) == False ,'F').otherwise('T'))

In [22]:
for col_name in ch_col:
    b_10 = b_10.withColumn(col_name,when(col(col_name) == False ,'F').otherwise('T'))

In [23]:
b_08 = b_08.withColumn("Inpatient_reimbursement_amount",when(col("Inpatient_reimbursement_amount") > 0 ,"Reimbursed").otherwise("Not Reimbursed"))
b_08 = b_08.withColumn("Gender", when(b_08["Gender"] == 1 , 'M').otherwise('F'))

In [24]:
b_09 = b_09.withColumn("Inpatient_reimbursement_amount",when(col("Inpatient_reimbursement_amount") > 0 ,"Reimbursed").otherwise("Not Reimbursed"))
b_09 = b_09.withColumn("Gender", when(b_09["Gender"] == 1 , 'M').otherwise('F'))

In [25]:
b_10 = b_10.withColumn("Inpatient_reimbursement_amount",when(col("Inpatient_reimbursement_amount") > 0 ,"Reimbursed").otherwise("Not Reimbursed"))
b_10 = b_10.withColumn("Gender", when(b_10["Gender"] == 1 , 'M').otherwise('F'))

In [26]:
b_08.dtypes

[('Age', 'int'),
 ('Alzheimer', 'string'),
 ('Cancer', 'string'),
 ('Chronic_Kidney_Disease', 'string'),
 ('Chronic_Obstructive_Pulmonary_Disease', 'string'),
 ('Depression', 'string'),
 ('Diabetes', 'string'),
 ('End_stage_renal_disease_Indicator', 'string'),
 ('Gender', 'string'),
 ('Heart_Failure', 'string'),
 ('Inpatient_reimbursement_amount', 'string'),
 ('Ischemic_Heart_Disease', 'string'),
 ('Osteoporosis', 'string'),
 ('Rheumatoid_arthritis_osteoarthritis', 'string'),
 ('Stroke_transient_Ischemic_Attack', 'string'),
 ('DESYNPUF_ID', 'string')]

In [27]:
ben = b_08.union(b_09).union(b_10)

In [28]:
ben.columns

['Age',
 'Alzheimer',
 'Cancer',
 'Chronic_Kidney_Disease',
 'Chronic_Obstructive_Pulmonary_Disease',
 'Depression',
 'Diabetes',
 'End_stage_renal_disease_Indicator',
 'Gender',
 'Heart_Failure',
 'Inpatient_reimbursement_amount',
 'Ischemic_Heart_Disease',
 'Osteoporosis',
 'Rheumatoid_arthritis_osteoarthritis',
 'Stroke_transient_Ischemic_Attack',
 'DESYNPUF_ID']

In [29]:
Inp.columns

['Beneficiary_Blood_Deductible_Liability_Amount',
 'Beneficiary_Inpatient_Deductible_Amount',
 'Beneficiary_PartA_Coinsurance_Liability_Amount',
 'Claim_Admission_Date',
 'Claim_Admitting_Diagnosis_Code',
 'Claim_End_Date',
 'Claim_ID',
 'Claim_Pass_Thru_Per_Diem_Amount',
 'Claim_Payment_Amount',
 'Claim_Start_Date',
 'Claim_Utilization_Day_Count',
 'DESYNPUF_ID',
 'Inpatient_discharged_date',
 'Primary_Payer_Claim_Paid_Amount',
 'Provider_Number',
 '_id']

In [30]:
col = ['Claim_Payment_Amount','Claim_Utilization_Day_Count','DESYNPUF_ID']

In [31]:
Inp = Inp.select(col)

In [32]:
Inp.columns

['Claim_Payment_Amount', 'Claim_Utilization_Day_Count', 'DESYNPUF_ID']

In [33]:
binp = ben.join(Inp, on="DESYNPUF_ID", how="inner")

In [34]:
binp.columns

['DESYNPUF_ID',
 'Age',
 'Alzheimer',
 'Cancer',
 'Chronic_Kidney_Disease',
 'Chronic_Obstructive_Pulmonary_Disease',
 'Depression',
 'Diabetes',
 'End_stage_renal_disease_Indicator',
 'Gender',
 'Heart_Failure',
 'Inpatient_reimbursement_amount',
 'Ischemic_Heart_Disease',
 'Osteoporosis',
 'Rheumatoid_arthritis_osteoarthritis',
 'Stroke_transient_Ischemic_Attack',
 'Claim_Payment_Amount',
 'Claim_Utilization_Day_Count']

In [35]:
binp_p = binp.toPandas()

In [36]:
binp_p.to_csv('C:/Users/vishw/OneDrive/Desktop/Insurance/ML/ML_dataset.csv',index=False)